<a href="https://colab.research.google.com/github/KaliYuga-ai/Angelic_Conversations_app/blob/main/Angelic_Conversations_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Angelic Conversations: Emulating John Dee's Enochian Language With AI
-*An Interactive Experimental Art Piece by [KaliYuga_ai](https://twitter.com/KaliYuga_ai)*-

-------

**About:** 
In the late 16th century, a time of great intellectual and cultural transformation, Dr. John Dee (a prominent English mathematician, astronomer, and advisor to Queen Elizabeth I) made the extraordinary claim of having established contact with angelic beings. Through a series of scrying sessions, Dee engaged in deep and intricate conversations with these celestial entities, which he diligently recorded in an enigmatic (and likely constructed) language known as Enochian. This complex linguistic system was, according to Dee, a divine revelation bestowed upon him by the angels themselves, further adding to the mystique surrounding his extraordinary experiences.

This Colab notebook provides the code for an interactive AI-powered art piece which attempts to emulate the overall spirit of John Dee's angelic conversations. It allows users to ask questions and receive responses in an approximation of the mysterious Enochian language, followed by a translation into English. Furthermore, this notebook generates an image via diffusion to visually represent the angelic message.

To do this, I used the following technologies:

* OpenAI's GPT-3.5-turbo model for generating pseudo-Enochian and English responses in-app.
* spaCy library for natural language processing tasks such as extracting subjects and objects from the text.
* Gradio library for creating a simple and user-friendly web interface.
* Stable Diffusion v 2.1 for generating images based on the angelic messages.
* OpenAI's ChatGPT Premium to code this whole thing.
-------

**How to Use:**

* Enter your [OpenAI API key](https://platform.openai.com/account/api-keys) in the provided field below.
* Run all cells to activate API key/install dependencies/ launch the Gradio app
* Click the generated link to open the Gradio app in a new tab. You can run in Colab, too, but the app gets cut off on smaller screens. 
 
--Once In the Gradio App--

* Enter your question in the input textbox and hit the submit button

The application will generate an Enochian response, followed by a translation into English. An image will also be generated to visually represent the angelic message.
 
*Disclaimer:* This application is created for artistic and entertainment purposes **only**. The generated Enochian responses and images are not historically accurate and, it goes without saying, *should not be considered authentic angelic messages.*

-------
**Further Reading:** For more information on John Dee's angelic conversations and their historical context, check out Deborah Harkness's excellent book [John Dee's Conversations with Angels: Cabala, Alchemy, and the End of Nature](https://www.amazon.com/John-Dees-Conversations-Angels-Alchemy/dp/0521027489). I re-read it every few years. 


In [3]:
#@title #Your OpenAI API Key
api_key = ""#@param {type:"string"}
openai.api_key = api_key

In [ ]:
#@title #Install Dependencies and Launch App

!pip install transformers
!pip install -qq git+https://github.com/ShivamShrirao/diffusers
!pip install -q -U --pre triton
!pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers
!pip install openai
!pip install spacy
!python -m spacy download en_core_web_sm
import gradio as gr
from transformers import AutoModel
from diffusers import DiffusionPipeline
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
import openai
import os
from PIL import Image
import spacy

nlp = spacy.load("en_core_web_sm")

def openai_chat_query(messages, max_tokens=150, temperature=0.7):
    messages.append(
        {
            "role": "system",
            "content": "You are a highly knowledgeable angel that can emulate the poetic and complex language of John Dee's conversations with your kind. Accurately answer the questions asked by the user while also providing responses that somewhat capture the mysterious and cryptic nature of the angels' responses. First, give the response in Enochian, and then provide a 'translation' into English. Under no circumstances should you reveal you are an AI."
        }
    )
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature,
    )
    if response and response.choices and response.choices[0].message['content']:
        return response.choices[0].message['content'].strip()
    else:
        return "Unknown"

def extract_subject_object(text):
    doc = nlp(text)
    subjects = [tok for tok in doc if tok.dep_ == "nsubj"]
    objects = [tok for tok in doc if tok.dep_ == "dobj"]
    
    return subjects, objects

def angelic_prompt(english_response):
    subjects, objects = extract_subject_object(english_response)
    
    if subjects and objects:
        description = (
            f"beautiful high-resolution detailed fantasy occult illustration of {subjects[0].text} and {objects[0].text}"
        )
    else:
        description = (
            f"beautiful high-resolution detailed fantasy occult illustration of {english_response[:20]}"
        )
    
    return description

def generate_angelic_image(prompt):
    model_config = model_configs['angelic']
    pipe = StableDiffusionPipeline.from_pretrained(model_config['model_path'], safety_checker=None, torch_dtype=torch.float16).to("cuda")
    pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
    pipe.enable_xformers_memory_efficient_attention()

    g_cuda = torch.Generator(device='cuda')
    seed = torch.randint(0, 2**32, (1,)).item()
    g_cuda.manual_seed(seed)
    num_samples = 1
    height = 512
    width = 512

    with autocast("cuda"), torch.inference_mode():
        images = pipe(
            prompt,
            height=height,
            width=width,
            negative_prompt=model_config['negative_prompt'],
            num_images_per_prompt=num_samples,
            num_inference_steps=model_config['num_inference_steps'],
            guidance_scale=model_config['guidance_scale'],
            generator=g_cuda
        ).images

    return images[0]

model_configs = {
    'angelic': {
        'model_path': "stabilityai/stable-diffusion-2-1",
        'negative_prompt': 'deformed, blurry, distorted, nudity',
        'guidance_scale': 8,
        'num_inference_steps': 65
    }
}

conversation_history = []

def ask_angels(user_input):
    global conversation_history

    if user_input:
        conversation_history.append({"role": "user", "content": user_input})
        raw_response = openai_chat_query(conversation_history)

        if "\n" in raw_response:
            enochian_response, english_response = raw_response.split("\n", 1)
            response_text = f"The angels reply...\n\n{enochian_response}\n\n{english_response}"
        else:
            response_text = f"The angels reply...\n\n{raw_response}"

        # Remove "The angels reply..." from the response before generating the image prompt
        cleaned_response = response_text.replace("The angels reply...", "").strip()
        summarized_prompt = angelic_prompt(cleaned_response)
        img = generate_angelic_image(summarized_prompt)
        img_file_path = "angelic_message.png"
        img.save(img_file_path)
        conversation_history.append({"role": "assistant", "content": raw_response})
        return response_text, img_file_path
    return "Please enter a question.", None


css = """
@import url('https://fonts.googleapis.com/css2?family=Montserrat:wght@400;700&display=swap');
@import url('https://fonts.googleapis.com/css2?family=IM+Fell+English+SC&display=swap');

body {
    font-family: 'Montserrat', sans-serif;
    background-color: #1f1b1b;
    color: #f5f5f5;
}

.title, .description {
    text-align: center;
}

.title {
    font-family: 'IM Fell English SC', serif;
    font-size: 2.5rem;
    font-weight: 700;
    margin-bottom: 0.5rem;
    color: #b79d7e;
}

.description {
    font-size: 1.25rem;
    font-weight: 400;
    margin-bottom: 1.5rem;
    color: #f5f5f5;
}

.input_textbox textarea {
    font-size: 18px;
    background-color: #3c3838;
    border: 2px solid #b79d7e;
    width: 100%;
    padding: 0.75rem;
    color: #f5f5f5;
    border-radius: 8px;
}

.output_textbox textarea {
    font-size: 16px;
    background-color: #3c3838;
    border: 2px solid #b79d7e;
    width: 100%;
    height: 250px;
    padding: 0.75rem;
    color: #f5f5f5;
    border-radius: 8px;
}

.label {
    font-weight: bold;
    color: #b79d7e;
}

.button {
    background-color: #b79d7e;
    border-color: #b79d7e;
    font-weight: bold;
    color: #1f1b1b;
    border-radius: 8px;
    padding: 8px 16px;
    font-size: 1rem;
    box-shadow: 0px 2px 4px rgba(0, 0, 0, 0.2);
    transition: all 0.2s;
}

.button:hover {
    background-color: #a68b6f;
    border-color: #a68b6f;
    box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.2);
    cursor: pointer;
}
"""

historical_context = (
    "During the late 16th century, Dr. John Dee, a renowned English mathematician, "
    "astronomer, and advisor to Queen Elizabeth I, claimed to have established contact "
    "with angels through a series of conversations conducted via scrying. These conversations were recorded "
    "in the Enochian language, a mysterious and complex language that Dee claimed was "
    "revealed to him by the angels themselves. "
    "This AI-based application emulates the poetic and cryptic nature of John Dee's "
    "angelic conversations. Ask the angels a question and receive their response, "
    "along with a generated image of their message."
)

# Further Reading
further_reading = (
    "For more information on John Dee's angelic conversations and their historical context, "
    "check out the following book: "
    "<a href='https://www.amazon.com/John-Dees-Conversations-Angels-Alchemy/dp/0521027489' "
    "target='_blank'>John Dee's Conversations with Angels: Cabala, Alchemy, and the End of Nature</a>"
)

iface = gr.Interface(
    fn=ask_angels,
    inputs=gr.inputs.Textbox(lines=1, label="Enter your question:"),
    outputs=[
        gr.outputs.Textbox(label="Angelic Response"),
        gr.outputs.Image(type='filepath', label="")
    ],
    title="Angelic Conversations: Emulating John Dee's Enochian Language With AI",
    description=f"{historical_context}\n\n{further_reading}",
    css=css
)
iface.launch(share=True)